<a href="https://colab.research.google.com/github/thinkdeepai/reinforcement-learning-training/blob/master/OpenAI_Gym_Taxi_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import gym
import sys
import math
import numpy as np
from collections import deque
from collections import defaultdict

In [0]:
def interact(env, agent, num_episodes=20000, window=100):
    """ Monitor agent's performance.
    
    Params
    ======
    - env: instance of OpenAI Gym's Taxi-v3 environment
    - agent: instance of class Agent (see Agent.py for details)
    - num_episodes: number of episodes of agent-environment interaction
    - window: number of episodes to consider when calculating average rewards

    Returns
    =======
    - avg_rewards: deque containing average rewards
    - best_avg_reward: largest value in the avg_rewards deque
    """
    # initialize average rewards
    avg_rewards = deque(maxlen=num_episodes)
    # initialize best average reward
    best_avg_reward = -math.inf
    # initialize monitor for most recent rewards
    samp_rewards = deque(maxlen=window)
    # for each episode
    for i_episode in range(1, num_episodes+1):
        # begin the episode
        state = env.reset()
        # initialize the sampled reward
        samp_reward = 0
        while True:
            # agent selects an action
            action = agent.select_action(state)
            # agent performs the selected action
            next_state, reward, done, _ = env.step(action)
            # agent performs internal updates based on sampled experience
            agent.step(state, action, reward, next_state, done)
            # update the sampled reward
            samp_reward += reward
            # update the state (s <- s') to next time step
            state = next_state
            if done:
                # save final sampled reward
                samp_rewards.append(samp_reward)
                break
        if (i_episode >= 100):
            # get average reward from last 100 episodes
            avg_reward = np.mean(samp_rewards)
            # append to deque
            avg_rewards.append(avg_reward)
            # update best average reward
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward
        # monitor progress
        print("\rEpisode {}/{} || Best average reward {}".format(i_episode, num_episodes, best_avg_reward), end="")
        sys.stdout.flush()
        # check if task is solved (according to OpenAI Gym)
        if best_avg_reward >= 9.7:
            print('\nEnvironment solved in {} episodes.'.format(i_episode), end="")
            break
        if i_episode == num_episodes: print('\n')
    return avg_rewards, best_avg_reward

In [0]:
class Agent:

    def __init__(self, nA=6):
        self.nA = nA
        self.Q = defaultdict(lambda: np.zeros(self.nA))
        self.epsilon = 0
        self.decay = 0
        self.gamma = 0
        self.alpha = 0
        self.epsilon_function = None;
        self.next_value_function = None;
        #best score for self.init_for_sarsa() is 9.261 | slower to learn that q-learning
        #self.init_for_sarsa()
        #best score for self.init_for_q_learning is 9.26
        self.init_for_q_learning()
        #fixed epsilon=0.005 with alpha = 1 and gamma = 1 was very bad with a score of 8.655
        #current settings succeeded with 9.253
        #self.init_for_expected_sarsa()
    def init_for_sarsa(self):
        self.epsilon = 1
        self.decay = 0.5
        #discounted return rate
        self.gamma = 0.9
        #tendency to use recent event
        self.alpha = 0.5
        self.epsilon_function = self.decayed_epsilon
        def calculate(next_state):
            return self.Q[next_state][self.select_action(next_state)]
        self.next_value_function = calculate 
    def init_for_q_learning(self):
        self.epsilon = 1
        self.decay = 0.9
        #discounted return rate
        self.gamma = 0.9
        #tendency to use recent event
        self.alpha = 0.3
        self.epsilon_function = self.decayed_min_epsilon
        def calculate(next_state):
            return np.max(self.Q[next_state])
        self.next_value_function = calculate
    def init_for_expected_sarsa(self):
        self.epsilon = 0.1
        self.decay = 0.5
        #discounted return rate
        self.gamma = 0.8
        #tendency to use recent event
        self.alpha = 0.7
        self.epsilon_function = self.fixed_epsilon
        def calculate(next_state):
            policy_s = self.get_action_probability(next_state)
            next_action_value = np.dot(self.Q[next_state], policy_s)
            return np.max(self.Q[next_state])
        self.next_value_function = calculate
    def decayed_min_epsilon(self):
        self.epsilon = max(self.epsilon*self.decay,0.01)
    def decayed_epsilon(self):
        self.epsilon *= self.decay
    def fixed_epsilon(self):
        pass
    def get_action_probability(self, state):
        policy_s = np.ones(self.nA) * self.epsilon / self.nA
        #find the best action and assign it the other side of the probabiliy epsilon
        best_action = np.argmax(self.Q[state])
        policy_s[best_action] += 1 - self.epsilon
        return policy_s
    def select_action(self, state):
        policy_s = self.get_action_probability(state)
        return np.random.choice(np.arange(self.nA), p=policy_s)
    
    def step(self, state, action, reward, next_state, done):
        self.epsilon_function()
        next_action_value = 0
        if not done:
            next_action_value = self.next_value_function(next_state)
        self.Q[state][action] += self.alpha * (reward  + self.gamma * next_action_value - self.Q[state][action])

In [0]:
env = gym.make('Taxi-v3')

## Sarsa

In [0]:
agent = Agent()
agent.init_for_sarsa()
avg_rewards, best_avg_reward = interact(env, agent)

Episode 20000/20000 || Best average reward 8.79



## Sarsa-max / q-learning

In [0]:
agent_qlearning = Agent()
agent.init_for_q_learning()
avg_rewards, best_avg_reward = interact(env, agent)

Episode 20000/20000 || Best average reward 8.53



## Expected Sarsa

In [0]:
agent_expected_sarsa = Agent()
agent.init_for_expected_sarsa()
avg_rewards, best_avg_reward = interact(env, agent)

Episode 20000/20000 || Best average reward 7.33

